In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [3]:
labels = df[1].values

In [4]:
type(labels)

numpy.ndarray

In [5]:
texts = df[0].values.tolist()

In [6]:
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification

In [7]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

C:\Users\yong\anaconda3\envs\textmining_cpu\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yong\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [8]:
from tensorflow.keras.utils import to_categorical
y_one_hot = to_categorical(labels)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(texts, y_one_hot, test_size=0.2, random_state=0)

In [10]:
X_train_tokenized = tokenizer(X_train, return_tensors="np", max_length=30, padding='max_length', truncation=True)
X_test_tokenized = tokenizer(X_test, return_tensors="np", max_length=30, padding='max_length', truncation=True)

In [11]:
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_projector', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_19', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [12]:
optimizer = tf.keras.optimizers.Adam(2e-5)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [13]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
checkpoint_filepath = "./checkpoints/checkpoint_distil_en"
mc = ModelCheckpoint(checkpoint_filepath, monitor='val_loss', mode='min', 
                     save_best_only=True, save_weights_only=True)

In [14]:
model.fit(dict(X_train_tokenized), y_train, epochs=10, batch_size=128, 
          validation_split=0.1, callbacks=[es, mc])

Epoch 1/10
39/39 [==============================] - 125s 3s/step - loss: 0.6160 - accuracy: 0.6680 - val_loss: 0.4445 - val_accuracy: 0.8303
Epoch 2/10
39/39 [==============================] - 117s 3s/step - loss: 0.3646 - accuracy: 0.8625 - val_loss: 0.3546 - val_accuracy: 0.8538
Epoch 3/10
39/39 [==============================] - 117s 3s/step - loss: 0.2386 - accuracy: 0.9189 - val_loss: 0.3215 - val_accuracy: 0.8827
Epoch 4/10
39/39 [==============================] - 108s 3s/step - loss: 0.1519 - accuracy: 0.9552 - val_loss: 0.3382 - val_accuracy: 0.8791
Epoch 5/10
39/39 [==============================] - 109s 3s/step - loss: 0.0996 - accuracy: 0.9741 - val_loss: 0.3991 - val_accuracy: 0.8736
Epoch 6/10
39/39 [==============================] - 109s 3s/step - loss: 0.0799 - accuracy: 0.9791 - val_loss: 0.4368 - val_accuracy: 0.8538
Epoch 6: early stopping


In [15]:
model.load_weights(checkpoint_filepath)

In [16]:
model.evaluate(dict(X_test_tokenized), y_test)

44/44 [==============================] - 10s 220ms/step - loss: 0.3178 - accuracy: 0.8786


[0.3178231120109558, 0.8786126971244812]

In [17]:
y_preds = model.predict(dict(X_test_tokenized))
prediction_probs = tf.nn.softmax(y_preds.logits,axis=1).numpy()
y_predictions = np.argmax(prediction_probs, axis=1)
y_test = np.argmax(y_test, axis=1)
from sklearn.metrics import classification_report
print(classification_report(y_predictions, y_test))

44/44 [==============================] - 11s 221ms/step
              precision    recall  f1-score   support

           0       0.89      0.86      0.87       668
           1       0.87      0.90      0.88       716

    accuracy                           0.88      1384
   macro avg       0.88      0.88      0.88      1384
weighted avg       0.88      0.88      0.88      1384

